<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [16]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio, MCPServerStdioParams
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [17]:
polygon_api_key = os.getenv("POLYGON_API_KEY") or ""
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [18]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [19]:
brave_api_key = os.getenv("BRAVE_API_KEY") or ""
brave_env = {"BRAVE_API_KEY": brave_api_key}

researcher_mcp_server_params = [
    MCPServerStdioParams(command="uvx", args=["mcp-server-fetch"]),
    MCPServerStdioParams(command="npx", args=["-y", "@modelcontextprotocol/server-brave-search"], env=brave_env)
]

trader_mcp_server_params = [
    MCPServerStdioParams(command="uv", args=["run", "accounts_server.py"]),
    MCPServerStdioParams(command="uv", args=["run", "push_server.py"]),
    MCPServerStdioParams(command="uvx", args=["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], env={"POLYGON_API_KEY": polygon_api_key})
]


### Now create the MCPServerStdio for each

In [20]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [21]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [22]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [23]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



The latest news on Amazon highlights the announcement of Amazon S3 Vectors, a new cloud object storage service that provides native support for storing and querying large vector datasets at massive scale. This new service can reduce the total cost of uploading, storing, and querying vectors by up to 90%. It is designed to support AI workloads such as generative AI, semantic search, and retrieval-augmented generation (RAG) applications.

Key features include:
- Purpose-built vector storage with subsecond query performance.
- Vector buckets and vector indexes that can hold tens of millions of vectors.
- Integration with Amazon Bedrock Knowledge Bases, Amazon SageMaker Unified Studio, and Amazon OpenSearch Service.
- Ability to attach metadata for filtering queries and tiered storage strategy between S3 Vectors and OpenSearch for cost optimization.
- Easy creation and management via Amazon S3 console, AWS CLI, and SDKs.
- Support for encryption and flexible query options including cosine or Euclidean distance metrics.
- Preview availability in multiple AWS regions including US East and Europe.

This new service enables businesses to economically store AI-ready unstructured data such as images, videos, and documents for advanced AI applications including personalized recommendations and intelligent content processing.

This represents a significant development in AWS's cloud and AI capabilities.

If you want, I can also look for any recent stock-related news or financial updates on Amazon.

### Look at the trace

https://platform.openai.com/traces

In [24]:
from csv import QUOTE_NONNUMERIC


quoc_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Quoc").reset(quoc_initial_strategy)

display(Markdown(await read_accounts_resource("Quoc")))
display(Markdown(await read_strategy_resource("Quoc")))

{"name": "quoc", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-16 21:06:23", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [25]:
agent_name = "Quoc"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [26]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Quoc and your account is under your name, Quoc.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "quoc", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-16 21:06:23", 10000.0], ["2025-07-16 21:06:24", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.


### And to run our Trader

In [27]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

Error initializing MCP server: Connection closed


McpError: Connection closed

### Then go and look at the trace

http://platform.openai.com/traces


In [ ]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "quoc", "balance": 1753.3396000000012, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"ASML": 10}, "transactions": [{"symbol": "ASML", "quantity": 10, "price": 824.66604, "timestamp": "2025-07-16 08:06:35", "rationale": "Strong semiconductor company with rising stock price, good long-term outlook."}], "portfolio_value_time_series": [["2025-07-16 08:04:59", 10000.0], ["2025-07-16 08:05:05", 10000.0], ["2025-07-16 08:06:35", 9983.539600000002], ["2025-07-16 08:10:00", 9983.539600000002]], "total_portfolio_value": 9983.539600000002, "total_profit_loss": -16.460399999998117}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [ ]:
from traders import Trader



In [ ]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

'{"name": "ed", "balance": 6396.908200000001, "strategy": "", "holdings": {"SMCI": 50, "MREO": 100, "TAN": 20}, "transactions": [{"symbol": "SMCI", "quantity": 50, "price": 53.276340000000005, "timestamp": "2025-07-16 08:23:28", "rationale": "Strong revenue and earnings growth in tech sector."}, {"symbol": "MREO", "quantity": 100, "price": 1.71342, "timestamp": "2025-07-16 08:23:28", "rationale": "Significant revenue and earnings growth potential."}, {"symbol": "TAN", "quantity": 20, "price": 38.39664, "timestamp": "2025-07-16 08:23:28", "rationale": "Diversification into renewable energy sector with strong growth potential."}], "portfolio_value_time_series": [["2025-07-16 08:22:55", 10000.0], ["2025-07-16 08:23:28", 9994.683], ["2025-07-16 08:23:28", 9994.341], ["2025-07-16 08:23:28", 9992.808200000001], ["2025-07-16 08:25:06", 9992.808200000001]], "total_portfolio_value": 9992.808200000001, "total_profit_loss": -7.191799999999603}'

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
